In [ ]:
import pyomo.environ as pyomo

# solver
solver = pyomo.SolverFactory('cbc',executable=r'C:\Cbc-master-win64-msvc16-mt\bin\cbc.exe')
solver.options['sec'] = 5

In [ ]:
# read data 
import pandas as pd 

df = pd.read_csv('pyomo_data.csv')
i_set = df['i'].unique()
t_set = df['t'].unique()

pf_df = pd.read_csv('pyomo_pf.csv')

buyer 

In [ ]:
# create a model 
model = pyomo.ConcreteModel()

# series 
model.nt = pyomo.Param(initialize=len(df))
model.T = pyomo.Set(initialize=t_set)
model.I = pyomo.Set(initialize=i_set)

# Parameters
model.C_geninv_it = pyomo.Param(model.I, model.T, initialize={(i, t): val for i, t, val in zip(df['i'], df['t'], df['C_geninv_it'])})  # C^{geninv}_{i,t}
model.C_gen_it = pyomo.Param(model.I, model.T, initialize={(i, t): val for i, t, val in zip(df['i'], df['t'], df['C_gen_it'])})        # C^{gen}_{i,t}
model.f_it = pyomo.Param(model.I, model.T, initialize={(i, t): val for i, t, val in zip(df['i'], df['t'], df['f_it'])})                # f_{i,t}
model.CO2_it = pyomo.Param(model.I, model.T, initialize = {(i, t): val for i, t, val in zip(df['i'], df['t'], df['CO2_it'])})          #CO2_{i,t}
model.Y_it = pyomo.Param(model.I, model.T, initialize = {(i, t): val for i, t, val in zip(df['i'], df['t'], df['Y_it'])})              #Y_{i,t}
model.PS_it = pyomo.Param(model.I, model.T, initialize ={(i, t): val for i, t, val in zip(df['i'], df['t'], df['PS_it'])})             # P^S_{i,t}
model.PF_i = pyomo.Param(model.I, initialize={i: val for i, val in zip(pf_df['i'], pf_df['PF_i'])})                                    # P^F_i

# Variables
model.g_it = pyomo.Var(model.I, model.T, domain=pyomo.NonNegativeReals)       # g_{i,t}
model.h_it = pyomo.Var(model.I, model.T, domain=pyomo.NonNegativeReals)       # h_{i,t}
model.QF_i = pyomo.Var(model.I, domain=pyomo.NonNegativeReals)                # Q^F_i
model.QS_it = pyomo.Var(model.I, model.T, domain=pyomo.NonNegativeReals)      # Q^S_{i,t}
model.D_it = pyomo.Var(model.I, model.T, domain=pyomo.NonNegativeReals)       #D_{i,t}

# Objective Function
def objective_func(model):
    return sum(model.PF_i[i] * model.QF_i[i] + \
        sum(model.C_geninv_it[i, t] * model.h_it[i, t] + model.C_gen_it[i, t] * model.g_it[i, t] \
            + model.PS_it[i, t] * model.QS_it[i, t] for t in model.T) 
        for i in model.I)
model.objective = pyomo.Objective(rule = objective_func, sense = pyomo.minimize)

# constraints 
def request(model, i):
    return sum(model.Y_it[i, t] for t in model.T) <= model.QF_i[i]
model.requestConstraint = pyomo.Constraint(model.I, rule=request)

def carbon_target(model, i, t):  
    return model.f_it[i, t] * model.g_it[i, t] <= model.CO2_it[i, t]
model.carbontargetConstraint = pyomo.Constraint(model.I, model.T, rule=carbon_target)

def demand(model,t): 
    return sum(model.Y_it[i, t] + model.g_it[i, t] + model.QS_it[i, t] - model.D_it[i, t] for i in model.I) == 0
model.demandConstraint = pyomo.Constraint(model.T, rule=demand)

def generator_capacity (model, i, t):  
    return model.g_it[i, t] <= sum(model.h_it[i, tp] for tp in model.T if tp <= t)
model.generatorConstraint = pyomo.Constraint(model.I, model.T, rule=generator_capacity)


solve

In [23]:
result = solver.solve(model)

print(result.solver.status)
print(result.solver.termination_condition)

print(model.objective())
print(model.display())

ok
optimal
11828.9035
Model unknown

  Variables:
    g_it : Size=120, Index=I*T
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
          (1, 1) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 2) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 3) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 4) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 5) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 6) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 7) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 8) :     0 :   0.0 :  None : False : False : NonNegativeReals
          (1, 9) :     0 :   0.0 :  None : False : False : NonNegativeReals
         (1, 10) :     0 :   0.0 :  None : False : False : NonNegativeReals
         (1, 11) :     0 :   0.0 :  None : False : False : NonNegativeReals
         (1, 12) 

In [22]:
for t in model.T:
    total_supplied = sum(model.QF_i[i]() for i in model.I) \
                     + sum(model.QS_it[i, t]() for i in model.I) \
                     + sum(model.g_it[i, t]() for i in model.I)
    demand = sum(model.D_it[i, t]() for i in model.I)
    print(f"Time {t}: Supply = {total_supplied}, Demand = {demand}")


Time 1: Supply = 1495.61, Demand = 140.11
Time 2: Supply = 1495.61, Demand = 127.91
Time 3: Supply = 1495.61, Demand = 106.48
Time 4: Supply = 1495.61, Demand = 114.64
Time 5: Supply = 1495.61, Demand = 138.98
Time 6: Supply = 1495.61, Demand = 111.48
Time 7: Supply = 1495.61, Demand = 128.47
Time 8: Supply = 1495.61, Demand = 124.05
Time 9: Supply = 1495.61, Demand = 130.05
Time 10: Supply = 1495.61, Demand = 128.26
Time 11: Supply = 1495.61, Demand = 121.83
Time 12: Supply = 1495.61, Demand = 123.35
